In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from helpers import *

In [2]:
#merging all the datasets into one
mergestorm_df = pd.concat( 
    map(pd.read_csv, ["resources/stormevents_2022.csv", 
                      "resources/stormevents_2021.csv",
                      "resources/stormevents_2020.csv",
                      "resources/stormevents_2019.csv",
                      "resources/stormevents_2018.csv", 
                      "resources/stormevents_2017.csv", 
                      "resources/stormevents_2016.csv", 
                      "resources/stormevents_2015.csv", 
                      "resources/stormevents_2014.csv", 
                      "resources/stormevents_2013.csv", 
                      "resources/stormevents_2012.csv", 
                      "resources/stormevents_2011.csv", 
                      "resources/stormevents_2010.csv", 
                      "resources/stormevents_2009.csv", 
                      "resources/stormevents_2008.csv"]), ignore_index=True) 
mergestorm_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,202202,20,2118,202202,20,2218,165464,999902,NEVADA,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strong winds increased ahead of an approaching...,"Station (UP994) 3.1 SE West Wendover, Elevatio...",CSV
1,202202,21,800,202202,22,1000,165465,999903,NEVADA,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,CSV
2,202202,22,200,202202,22,900,165465,999904,NEVADA,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A low centered over northern and central Nevad...,Fifteen inches fell at station (TJMN2) Toe Jam...,CSV
3,202202,18,1609,202202,18,1609,165611,1001181,ATLANTIC SOUTH,87,...,7.0,SE,PONTE VEDRA,30.050,-81.1700,30.0500,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,CSV
4,202202,2,0,202202,3,0,165668,1001527,AMERICAN SAMOA,97,...,5.0,NNW,VAITOGI,-14.333,-170.7157,-14.3393,-170.7268,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",CSV


In [7]:
# Create a DataFrame with selected columns 
clean_mergestorm_df = mergestorm_df[["YEAR","EPISODE_ID", "EVENT_ID", "STATE", "EVENT_TYPE",  
                                "INJURIES_DIRECT", "INJURIES_INDIRECT","DEATHS_DIRECT", "DEATHS_INDIRECT",
                                "DAMAGE_PROPERTY","DAMAGE_CROPS", "SOURCE", "MAGNITUDE", "MAGNITUDE_TYPE", 
                                "TOR_F_SCALE", "TOR_LENGTH", "TOR_WIDTH", "TOR_OTHER_CZ_STATE", "BEGIN_LOCATION", 
                                "BEGIN_LAT", "BEGIN_LON", "END_LAT", "END_LON"]]
clean_mergestorm_df.head()

,YEAR,EPISODE_ID,EVENT_ID,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,...,MAGNITUDE_TYPE,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
0,2022,165464,999902,NEVADA,High Wind,0,0,0,0,0.00K,...,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,165465,999903,NEVADA,Heavy Snow,0,0,0,0,0.00K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,165465,999904,NEVADA,Heavy Snow,0,0,0,0,0.00K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,165611,1001181,ATLANTIC SOUTH,Waterspout,0,0,0,0,0.00K,...,NaN,NaN,NaN,NaN,NaN,PONTE VEDRA,30.050,-81.1700,30.0500,-81.1700
4,2022,165668,1001527,AMERICAN SAMOA,Heavy Rain,0,0,0,0,50.00K,...,NaN,NaN,NaN,NaN,NaN,VAITOGI,-14.333,-170.7157,-14.3393,-170.7268


In [8]:
clean_mergestorm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948286 entries, 0 to 948285
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   YEAR                948286 non-null  int64  
 1   EPISODE_ID          948286 non-null  int64  
 2   EVENT_ID            948286 non-null  int64  
 3   STATE               948286 non-null  object 
 4   EVENT_TYPE          948286 non-null  object 
 5   INJURIES_DIRECT     948286 non-null  int64  
 6   INJURIES_INDIRECT   948286 non-null  int64  
 7   DEATHS_DIRECT       948286 non-null  int64  
 8   DEATHS_INDIRECT     948286 non-null  int64  
 9   DAMAGE_PROPERTY     777435 non-null  object 
 10  DAMAGE_CROPS        775903 non-null  object 
 11  SOURCE              948286 non-null  object 
 12  MAGNITUDE           500839 non-null  float64
 13  MAGNITUDE_TYPE      340870 non-null  object 
 14  TOR_F_SCALE         21140 non-null   object 
 15  TOR_LENGTH          21140 non-null

In [22]:
#convert property damages from object to string 
retype_damage_col(clean_mergestorm_df)
clean_mergestorm_df.info()

TypeError: 'float' object is not subscriptable

In [25]:
clean_mergestorm_df["DAMAGE_PROPERTY"].value_counts()

DAMAGE_PROPERTY
0.00K      562766
nan        170851
1.00K       29178
5.00K       26815
10.00K      22795
            ...  
18.88M          1
2.73M           1
360.00M         1
116.40M         1
4.68M           1
Name: count, Length: 1784, dtype: int64

In [26]:
# Add new columns that combine direct and indirect deaths; direct and indirect injuries; and property damage and crop damage
clean_mergestorm_df.loc[:,"TOTAL DEATHS"] = clean_mergestorm_df["DEATHS_DIRECT"] + clean_mergestorm_df["DEATHS_INDIRECT"]
clean_mergestorm_df.loc[:,"TOTAL INJURIES"] = clean_mergestorm_df["INJURIES_DIRECT"] + clean_mergestorm_df["INJURIES_INDIRECT"]
# clean_mergestorm_df["TOTAL DAMAGES"] = clean_mergestorm_df["DAMAGE_PROPERTY"] + clean_mergestorm_df["DAMAGE_CROPS"]

clean_mergestorm_df.tail()

C:\Users\juggl\AppData\Local\Temp\ipykernel_29120\3569653725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df.loc[:,"TOTAL DEATHS"] = clean_mergestorm_df["DEATHS_DIRECT"] + clean_mergestorm_df["DEATHS_INDIRECT"]
C:\Users\juggl\AppData\Local\Temp\ipykernel_29120\3569653725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df.loc[:,"TOTAL INJURIES"] = clean_mergestorm_df["INJURIES_DIRECT"] + clean_mergestorm_df["INJURIES_INDIRECT"]


,YEAR,EPISODE_ID,EVENT_ID,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,...,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,TOTAL DEATHS,TOTAL INJURIES
948281,2008,18919,111312,FLORIDA,Hail,0,0,0,0,0.00K,...,NaN,NaN,NaN,RITAL,28.5233,-82.2320,28.5233,-82.2320,0,0
948282,2008,18495,108243,FLORIDA,Thunderstorm Wind,0,0,0,0,0.50K,...,NaN,NaN,NaN,HULL,27.1323,-81.9050,27.1323,-81.9050,0,0
948283,2008,18708,109851,GULF OF MEXICO,Marine Thunderstorm Wind,0,0,0,0,0.00K,...,NaN,NaN,NaN,MIDDLE TAMPA BAY,27.7651,-82.6270,27.7651,-82.6270,0,0
948284,2008,16372,94501,FLORIDA,Hail,0,0,0,0,0.00K,...,NaN,NaN,NaN,THONOTOSASSA,28.0762,-82.2492,28.0762,-82.2492,0,0
948285,2008,18495,108160,FLORIDA,Lightning,0,0,0,0,0.50K,...,NaN,NaN,NaN,SAFETY HARBOR,27.9824,-82.7087,27.9824,-82.7087,0,0


In [27]:
drop_merged_df = clean_mergestorm_df.drop(columns=["INJURIES_DIRECT", "INJURIES_INDIRECT","DEATHS_DIRECT", "DEATHS_INDIRECT"])
drop_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948286 entries, 0 to 948285
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   YEAR                948286 non-null  int64  
 1   EPISODE_ID          948286 non-null  int64  
 2   EVENT_ID            948286 non-null  int64  
 3   STATE               948286 non-null  object 
 4   EVENT_TYPE          948286 non-null  object 
 5   DAMAGE_PROPERTY     948286 non-null  object 
 6   DAMAGE_CROPS        948286 non-null  float64
 7   SOURCE              948286 non-null  object 
 8   MAGNITUDE           500839 non-null  float64
 9   MAGNITUDE_TYPE      340870 non-null  object 
 10  TOR_F_SCALE         21140 non-null   object 
 11  TOR_LENGTH          21140 non-null   float64
 12  TOR_WIDTH           21140 non-null   float64
 13  TOR_OTHER_CZ_STATE  2733 non-null    object 
 14  BEGIN_LOCATION      588860 non-null  object 
 15  BEGIN_LAT           588860 non-nul

In [33]:
drop_merged_df[["TOTAL DEATHS", "TOTAL INJURIES"]].value_counts()

TOTAL DEATHS  TOTAL INJURIES
0             0                 934362
1             0                   4554
0             1                   3415
              2                   1206
2             0                    697
                                 ...  
              175                    1
              250                    1
              353                    1
3             11                     1
161           1150                   1
Name: count, Length: 349, dtype: int64

In [42]:
print(clean_mergestorm_df["DEATHS_DIRECT"].sum()+clean_mergestorm_df["DEATHS_INDIRECT"].sum()  )
print(clean_mergestorm_df["INJURIES_DIRECT"].sum()+clean_mergestorm_df["INJURIES_INDIRECT"].sum()  )
print(drop_merged_df[["TOTAL DEATHS", "TOTAL INJURIES"]].sum())

12198
48392
TOTAL DEATHS      12198
TOTAL INJURIES    48392
dtype: int64


In [44]:
# Narrow down events that have had at least one death and/or at least one injury 
narrow_df = drop_merged_df.loc[(drop_merged_df["TOTAL DEATHS"] > 0) | (drop_merged_df["TOTAL INJURIES"] > 0)]

# reset the index 
narrow_df.reset_index(drop=True, inplace=True)

# Display sample data
narrow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13924 entries, 0 to 13923
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YEAR                13924 non-null  int64  
 1   EPISODE_ID          13924 non-null  int64  
 2   EVENT_ID            13924 non-null  int64  
 3   STATE               13924 non-null  object 
 4   EVENT_TYPE          13924 non-null  object 
 5   DAMAGE_PROPERTY     13924 non-null  object 
 6   DAMAGE_CROPS        13924 non-null  float64
 7   SOURCE              13924 non-null  object 
 8   MAGNITUDE           2811 non-null   float64
 9   MAGNITUDE_TYPE      2703 non-null   object 
 10  TOR_F_SCALE         1462 non-null   object 
 11  TOR_LENGTH          1462 non-null   float64
 12  TOR_WIDTH           1462 non-null   float64
 13  TOR_OTHER_CZ_STATE  461 non-null    object 
 14  BEGIN_LOCATION      6307 non-null   object 
 15  BEGIN_LAT           6307 non-null   float64
 16  BEGI

In [ ]:
# Export the narrowed data into a csv
narrow_df.to_csv("narrow_df.csv")



In [ ]:
# Read saved data
pd.read_csv("narrow_df.csv")

# Display sample data
narrow_df.info()

In [45]:
reordernarrow_df = narrow_df[["YEAR","EPISODE_ID", "EVENT_ID", "STATE", "EVENT_TYPE",  
                              "TOTAL DEATHS", "TOTAL INJURIES",
                              "DAMAGE_PROPERTY","DAMAGE_CROPS", "SOURCE", "MAGNITUDE", "MAGNITUDE_TYPE", "TOR_F_SCALE", "TOR_LENGTH", "TOR_WIDTH", "TOR_OTHER_CZ_STATE", "BEGIN_LOCATION", "BEGIN_LAT", "BEGIN_LON", "END_LAT", "END_LON"]]
reordernarrow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13924 entries, 0 to 13923
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YEAR                13924 non-null  int64  
 1   EPISODE_ID          13924 non-null  int64  
 2   EVENT_ID            13924 non-null  int64  
 3   STATE               13924 non-null  object 
 4   EVENT_TYPE          13924 non-null  object 
 5   TOTAL DEATHS        13924 non-null  int64  
 6   TOTAL INJURIES      13924 non-null  int64  
 7   DAMAGE_PROPERTY     13924 non-null  object 
 8   DAMAGE_CROPS        13924 non-null  float64
 9   SOURCE              13924 non-null  object 
 10  MAGNITUDE           2811 non-null   float64
 11  MAGNITUDE_TYPE      2703 non-null   object 
 12  TOR_F_SCALE         1462 non-null   object 
 13  TOR_LENGTH          1462 non-null   float64
 14  TOR_WIDTH           1462 non-null   float64
 15  TOR_OTHER_CZ_STATE  461 non-null    object 
 16  BEGI

In [53]:
sorted_df =reordernarrow_df.sort_values(["TOTAL DEATHS","TOTAL INJURIES"], ascending=False)
sorted_df

,YEAR,EPISODE_ID,EVENT_ID,STATE,EVENT_TYPE,TOTAL DEATHS,TOTAL INJURIES,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,...,MAGNITUDE_TYPE,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
9652,2011,49972,296617,MISSOURI,Tornado,161,1150,2.80B,0.0,NWS Storm Survey,...,NaN,EF5,8.8,1600.0,MO,CENTRAL CITY,37.0560,-94.5701,37.0544,-94.4105
1098,2021,158241,961302,OREGON,Excessive Heat,93,0,50.00K,50000.0,ASOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3640,2018,131864,788675,CALIFORNIA,Wildfire,86,12,17.00B,0.0,Fire Department/Rescue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,2022,174632,1058654,FLORIDA,Hurricane,60,0,7.00B,0.0,Official NWS Observations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2022,171258,1039790,TEXAS,Heat,53,0,0.00K,0.0,Broadcast Media,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13904,2008,17046,98570,FLORIDA,Lightning,0,1,0.00K,0.0,Broadcast Media,...,NaN,NaN,NaN,NaN,NaN,GAINESVILLE NORTH,29.6700,-82.3400,29.6700,-82.3400
13907,2008,17418,106822,IOWA,Thunderstorm Wind,0,1,10.00K,0.0,Newspaper,...,EG,NaN,NaN,NaN,NaN,MILLERSBURG,41.5495,-92.1974,41.5495,-92.1974
13909,2008,18506,108216,MISSOURI,Hail,0,1,0.00K,0.0,Emergency Manager,...,NaN,NaN,NaN,NaN,NaN,SCHELL CITY,38.0200,-94.1200,38.0200,-94.1200
13913,2008,21508,127149,FLORIDA,Lightning,0,1,0.00K,0.0,Newspaper,...,NaN,NaN,NaN,NaN,NaN,INDIAN RIVER SHRS,27.6700,-80.3800,27.6700,-80.3800


In [60]:
test = reordernarrow_df[["EPISODE_ID","EVENT_TYPE","STATE","TOTAL DEATHS", "TOTAL INJURIES"]].groupby("EPISODE_ID").sum()
test.sort_values(["TOTAL DEATHS","TOTAL INJURIES"], ascending=False).head(30)

,EVENT_TYPE,STATE,TOTAL DEATHS,TOTAL INJURIES
EPISODE_ID,,,,
49972,TornadoLightningTornado,MISSOURIMISSOURIMISSOURI,162,1153
50455,TornadoTornadoTornadoTornadoTornadoTornadoTorn...,ALABAMAALABAMAALABAMAALABAMAALABAMAALABAMAALAB...,149,2001
158241,Excessive HeatExcessive HeatExcessive HeatExce...,OREGONOREGONOREGONOREGONOREGONOREGONOREGON,118,0
50516,TornadoTornadoTornadoTornadoTornadoTornadoTorn...,ALABAMAALABAMAALABAMAALABAMAALABAMAALABAMAALAB...,100,124
174632,HurricaneHurricaneHurricaneHurricaneTropical S...,FLORIDAFLORIDAFLORIDAFLORIDAFLORIDAFLORIDAFLOR...,90,0
131864,Wildfire,CALIFORNIA,86,12
175040,Excessive HeatExcessive HeatExcessive HeatExce...,ARIZONAARIZONAARIZONAARIZONAARIZONAARIZONAARIZONA,60,0
156251,Cold/Wind ChillCold/Wind ChillExtreme Cold/Win...,TEXASTEXASTEXASTEXASTEXASTEXASTEXASTEXAS,59,0
164438,TornadoTornadoTornadoTornadoTornadoTornadoTorn...,KENTUCKYKENTUCKYKENTUCKYKENTUCKYKENTUCKYKENTUC...,58,519


In [49]:
groupbynarrow_df=reordernarrow_df.sort_values(["TOTAL DEATHS","TOTAL INJURIES"], ascending=False).groupby("TOTAL DEATHS").head()
groupbynarrow_df.head()

,YEAR,EPISODE_ID,EVENT_ID,STATE,EVENT_TYPE,TOTAL DEATHS,TOTAL INJURIES,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,...,MAGNITUDE_TYPE,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
9652,2011,49972,296617,MISSOURI,Tornado,161,1150,2.80B,0.0,NWS Storm Survey,...,NaN,EF5,8.8,1600.0,MO,CENTRAL CITY,37.056,-94.5701,37.0544,-94.4105
1098,2021,158241,961302,OREGON,Excessive Heat,93,0,50.00K,50000.0,ASOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3640,2018,131864,788675,CALIFORNIA,Wildfire,86,12,17.00B,0.0,Fire Department/Rescue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,2022,174632,1058654,FLORIDA,Hurricane,60,0,7.00B,0.0,Official NWS Observations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2022,171258,1039790,TEXAS,Heat,53,0,0.00K,0.0,Broadcast Media,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
groupbynarrow_df.loc[groupbynarrow_df["TOTAL DEATHS"] == 0,:]

AttributeError: 'DataFrameGroupBy' object has no attribute 'loc'

In [ ]:
# Create a scatter plot for total deaths over the years 
plt.figure(figsize=(12, 6))
plt.scatter(groupbynarrow_df['YEAR'], groupbynarrow_df['TOTAL DEATHS'], alpha=0.5)
plt.title('Total Deaths over the Years')
plt.xlabel('YEAR')
plt.ylabel('TOTAL DEATHS')
plt.grid(True)
plt.show()


In [ ]:
# Create a scatter plot for total injuries over the years 
plt.figure(figsize=(12, 6))
plt.scatter(groupbynarrow_df['YEAR'], groupbynarrow_df['TOTAL INJURIES'])
plt.title('Total Injuries over the Years')
plt.xlabel('YEAR')
plt.ylabel('TOTAL INJURIES')
plt.grid(True)
plt.show()

In [ ]:
drop_merged_df.head()

In [ ]:
groupbynarrow_df